In [1]:
!pip install mediapipe opencv-python filterpy scipy matplotlib


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:

import cv2
import mediapipe as mp
import numpy as np
from filterpy.kalman import KalmanFilter
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt

def preprocess_video(video_path, target_fps=30, target_size=(1280, 720), background_subtraction=False):
    
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error opening video: {video_path}")
        return []

    orig_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_interval = int(orig_fps / target_fps) if orig_fps > target_fps else 1

    if background_subtraction:
        back_sub = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=16, detectShadows=True)
        # or
        # back_sub = cv2.createBackgroundSubtractorKNN()

    frames = []
    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_idx % frame_interval == 0:
            # Resize frame
            frame_resized = cv2.resize(frame, target_size)

            # Convert to HSV for color normalization if needed or keep in BGR
            # Histogram equalization per channel
            frame_yuv = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2YUV)
            frame_yuv[:, :, 0] = cv2.equalizeHist(frame_yuv[:, :, 0])  
            frame_eq = cv2.cvtColor(frame_yuv, cv2.COLOR_YUV2BGR)

            # Background subtraction
            if background_subtraction:
                fg_mask = back_sub.apply(frame_eq)
                frame_eq = cv2.bitwise_and(frame_eq, frame_eq, mask=fg_mask)

            frames.append(frame_eq)

        frame_idx += 1

    cap.release()
    return frames

# Example 
# video_path = r"D:\fitz\fitzpatrick17k-main\pose\UCF50\Drumming\v_Drumming_g01_c01.avi"
video_path = r"D:\fitz\fitzpatrick17k-main\pose\UCF50\WalkingWithDog\v_WalkingWithDog_g01_c01.avi"
# video_path = r"D:\fitz\fitzpatrick17k-main\pose\UCF50\PullUps\v_Pullup_g01_c01.avi"

frames = preprocess_video(video_path, background_subtraction=True)
print(f"Total frames extracted: {len(frames)}")

# Optionally display first frame for verification
if frames:
    cv2.imshow("Preprocessed Frame", frames[0])
    cv2.waitKey(0)
    cv2.destroyAllWindows()


Total frames extracted: 240


In [3]:

def create_kalman_filter():
    kf = KalmanFilter(dim_x=4, dim_z=2)
    kf.F = np.array([[1, 0, 1, 0],
                     [0, 1, 0, 1],
                     [0, 0, 1, 0],
                     [0, 0, 0, 1]])
    kf.H = np.array([[1, 0, 0, 0],
                     [0, 1, 0, 0]])
    kf.P *= 1000
    kf.R = np.eye(2) * 5
    kf.Q = np.eye(4)
    return kf


In [4]:

def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-6)
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)


In [ ]:
last_state = 'STATIC'
static_count = 0

def classify_joint_movement(delta_y, angle_velocity, epsilon_y=5, epsilon_theta=2, static_threshold=3):
    global last_state, static_count
    
    if delta_y < -epsilon_y and angle_velocity > epsilon_theta:
        last_state = 'UP'
        static_count = 0
        return 'UP'
    elif delta_y > epsilon_y and angle_velocity < -epsilon_theta:
        last_state = 'DOWN'
        static_count = 0
        return 'DOWN'
    elif abs(delta_y) < epsilon_y and abs(angle_velocity) < epsilon_theta:
        static_count += 1
        if static_count >= static_threshold:
            return 'STATIC'
        else:
            return last_state
    else:
        static_count = 0
        return 'TRANSITION'


In [ ]:
# Mapping for different body parts: (name, joint triplet IDs)
JOINT_TRIPLETS = {
    "right_elbow": (12, 14, 16),  # Right shoulder, right elbow, right wrist
    "left_elbow": (11, 13, 15),   # Left shoulder, left elbow, left wrist
    "right_knee": (24, 26, 28),   # Right hip, right knee, right ankle
    "left_knee": (23, 25, 27)     # Left hip, left knee, left ankle
}

def process_joint_movements(image, joint_history, angle_history):
    y_offset = 50  # Starting position for text
    for name, (A_id, B_id, C_id) in JOINT_TRIPLETS.items():
        # Get latest coordinates
        if len(joint_history.get(f'joint_{B_id}', [])) < 2:
            continue 

        A = joint_history[f'joint_{A_id}'][-1]
        B = joint_history[f'joint_{B_id}'][-1]
        C = joint_history[f'joint_{C_id}'][-1]

        # Calculate angle
        angle = calculate_angle(A, B, C)
        angle_history.setdefault(name, []).append(angle)

        # Calculate movement parameters
        delta_y = B[1] - joint_history[f'joint_{B_id}'][-2][1]
        angle_velocity = 0
        if len(angle_history[name]) > 1:
            angle_velocity = angle - angle_history[name][-2]

        # Classify movement
        label = classify_joint_movement(delta_y, angle_velocity)

        # Display on image
        cv2.putText(
            image,
            f"{name.replace('_', ' ').title()}: {label} Angle: {angle:.2f}",
            (30, y_offset),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.9,
            (255, 0, 0),
            2
        )
        y_offset += 40  


In [7]:

# Define joint index mappings (Mediapipe Pose IDs)
ANGLE_JOINTS = {
    "right_elbow": (12, 14, 16),  # Shoulder, Elbow, Wrist
    "left_elbow": (11, 13, 15),
    "right_knee": (24, 26, 28),   # Hip, Knee, Ankle
    "left_knee": (23, 25, 27),
    # "spine": (11, 23, 24)         # Left Shoulder, Left Hip, Right Hip
    "left_hip_knee": (11, 23, 25),     # Left Shoulder, Left Hip, Left Knee
    "right_hip_knee": (12, 24, 26) 
}

# Colors for each joint label
JOINT_COLORS = {
    "right_elbow": (255, 0, 0),
    "left_elbow": (0, 255, 0),
    "right_knee": (0, 0, 255),
    "left_knee": (255, 255, 0),
    "left_hip_knee": (255, 0, 255),
    "right_hip_knee": (0, 255, 255)
}




In [ ]:
import csv

def draw_trajectories(image, joint_history, max_len=20):
    """Draw trailing lines showing movement history for each joint."""
    for key, points in joint_history.items():
        if len(points) < 2:
            continue
        color = (0, 255, 255)  # Example trail color: cyan
        for i in range(max(0, len(points)-max_len), len(points)-1):
            cv2.line(image, points[i], points[i+1], color, 2)




def save_csv_log(filename, angle_history, joint_history, status_history):
    """Save angle, joint positions, and status to CSV file."""
    with open(filename, mode='w', newline='') as csv_file:
        fieldnames = ['frame', 'joint', 'angle', 'x', 'y', 'status']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        frames = len(next(iter(angle_history.values()), []))
        for frame_idx in range(frames):
            for joint_name in ANGLE_JOINTS.keys():
                angle = angle_history.get(joint_name, [None]*frames)[frame_idx]
                # Use last known joint b (middle joint) position as example
                b_idx = ANGLE_JOINTS[joint_name][1]
                pos = joint_history.get(f'joint_{b_idx}', [(None, None)]*frames)[frame_idx]
                status = status_history.get(joint_name, [""]*frames)[frame_idx]
                writer.writerow({
                    'frame': frame_idx,
                    'joint': joint_name,
                    'angle': angle,
                    'x': pos[0],
                    'y': pos[1],
                    'status': status
                })


def write_video(output_path, frame_width, frame_height, fps=30):
    """Return a VideoWriter object for saving output."""
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    return cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))


In [9]:
pose_kalman_filters = {}
joint_history = {}
angle_history = {}
frame_height, frame_width = frames[0].shape[:2]
output_video_path = "output_annotated.avi"
video_writer = write_video(output_video_path, frame_width, frame_height, fps=30)
status_history = {joint: [] for joint in ANGLE_JOINTS.keys()}

In [26]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

video_path = r"D:\fitz\fitzpatrick17k-main\pose\UCF50\GolfSwing\v_GolfSwing_g01_c01.avi"

# video_path = r"D:\fitz\fitzpatrick17k-main\pose\UCF50\PullUps\v_Pullup_g01_c01.avi"
frames = preprocess_video(video_path, background_subtraction=False)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    for frame in frames:
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
        )

        if results.pose_landmarks:
            for idx, lm in enumerate(results.pose_landmarks.landmark):
                key = f'joint_{idx}'
                x, y = int(lm.x * image.shape[1]), int(lm.y * image.shape[0])

                if key not in pose_kalman_filters:
                    pose_kalman_filters[key] = create_kalman_filter()
                    # pose_kalman_filters[key].statePre = np.array([[x], [y], [0], [0]], np.float32)
                    pose_kalman_filters[key].x[:2] = np.array([[x], [y]])

                kf = pose_kalman_filters[key]
                kf.predict()
                # kf.correct(np.array([[np.float32(x)], [np.float32(y)]]))
                kf.update(np.array([x, y]))
                pred_x, pred_y = int(kf.x[0]), int(kf.x[1])

                if key not in joint_history:
                    joint_history[key] = []
                joint_history[key].append((pred_x, pred_y))

            # Process angles & movements (as you have it)
            for name, (a_idx, b_idx, c_idx) in ANGLE_JOINTS.items():
                if all(len(joint_history.get(f'joint_{i}', [])) >= 2 for i in (a_idx, b_idx, c_idx)):
                    a = joint_history[f'joint_{a_idx}'][-1]
                    b = joint_history[f'joint_{b_idx}'][-1]
                    c = joint_history[f'joint_{c_idx}'][-1]

                    angle = calculate_angle(a, b, c)
                    angle_history.setdefault(name, []).append(angle)

                    delta_y = b[1] - joint_history[f'joint_{b_idx}'][-2][1]
                    angle_velocity = angle - angle_history[name][-2] if len(angle_history[name]) > 1 else 0

                    label = classify_joint_movement(delta_y, angle_velocity)

                    cv2.putText(
                        image,
                        f"{name.replace('_', ' ').title()}: {label} {int(angle)}°",
                        (b[0] + 10, b[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6,
                        JOINT_COLORS[name], 2
                    )
                    status_history[name].append(label)
        posture = detect_posture(angle_history)
        cv2.putText(
            image,
            f"Posture: {posture}",
            (30, 30),  # top-left corner
            cv2.FONT_HERSHEY_SIMPLEX, 1.0,
            (0, 255, 255), 2
        )
        # draw_trajectories(image, joint_history)


        cv2.imshow('Pose Tracking', image)
        video_writer.write(image)

        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

cv2.destroyAllWindows()


C:\Users\lenovo\AppData\Local\Temp\ipykernel_14448\2424792725.py:43: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_x, pred_y = int(kf.x[0]), int(kf.x[1])


In [14]:

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
        )

        if results.pose_landmarks:
            for idx, lm in enumerate(results.pose_landmarks.landmark):
                key = f'joint_{idx}'
                x, y = int(lm.x * frame.shape[1]), int(lm.y * frame.shape[0])

                if key not in pose_kalman_filters:
                    pose_kalman_filters[key] = create_kalman_filter()
                    pose_kalman_filters[key].x[:2] = np.array([[x], [y]])

                kf = pose_kalman_filters[key]
                kf.predict()
                kf.update(np.array([x, y]))
                pred_x, pred_y = int(kf.x[0]), int(kf.x[1])

                if key not in joint_history:
                    joint_history[key] = []
                joint_history[key].append((pred_x, pred_y))
                # cv2.circle(image, (pred_x, pred_y), 4, (0, 255, 0), -1)

            # Process angles & movements
            for name, (a_idx, b_idx, c_idx) in ANGLE_JOINTS.items():
                if all(len(joint_history.get(f'joint_{i}', [])) >= 2 for i in (a_idx, b_idx, c_idx)):
                    a = joint_history[f'joint_{a_idx}'][-1]
                    b = joint_history[f'joint_{b_idx}'][-1]
                    c = joint_history[f'joint_{c_idx}'][-1]

                    angle = calculate_angle(a, b, c)
                    angle_history.setdefault(name, []).append(angle)

                    delta_y = b[1] - joint_history[f'joint_{b_idx}'][-2][1]
                    angle_velocity = angle - angle_history[name][-2] if len(angle_history[name]) > 1 else 0

                    label = classify_joint_movement(delta_y, angle_velocity)

                    cv2.putText(
                        image,
                        f"{name.replace('_', ' ').title()}: {label} {int(angle)}°",
                        (b[0] + 10, b[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6,
                        JOINT_COLORS[name], 2
                    )

        cv2.imshow('Pose Tracking', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()



C:\Users\lenovo\AppData\Local\Temp\ipykernel_32584\2059409268.py:37: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_x, pred_y = int(kf.x[0]), int(kf.x[1])


# Model Training

In [ ]:
import os
import cv2
import mediapipe as mp
import numpy as np
from tqdm import tqdm

# ====== Your existing imports/functions should be above ======
# Assuming you already have: preprocess_video, create_kalman_filter, calculate_angle, classify_joint_movement, ANGLE_JOINTS

mp_pose = mp.solutions.pose

# Output arrays
all_sequences = []
all_labels = []

# Root dataset path
DATASET_PATH = r"train"  # change to your dataset folder
actions = os.listdir(DATASET_PATH)  # ["Fall Down", "Lying Down", ...]

pose_kalman_filters = {}
joint_history = {}
angle_history = {}
status_history = {}

for action in actions:
    action_path = os.path.join(DATASET_PATH, action)
    videos = [f for f in os.listdir(action_path) if f.lower().endswith(('.mp4', '.avi'))]

    for video_file in tqdm(videos, desc=f"Processing {action}"):
        video_path = os.path.join(action_path, video_file)

        # Preprocess frames
        frames = preprocess_video(video_path, background_subtraction=False)

        sequence_features = []  # one sequence per video

        with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
            for frame in frames:
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = pose.process(image)

                if results.pose_landmarks:
                    frame_features = []
                    for idx, lm in enumerate(results.pose_landmarks.landmark):
                        x, y = int(lm.x * frame.shape[1]), int(lm.y * frame.shape[0])

                        key = f'joint_{idx}'
                        if key not in pose_kalman_filters:
                            pose_kalman_filters[key] = create_kalman_filter()
                            pose_kalman_filters[key].x[:2] = np.array([[x], [y]])
                        kf = pose_kalman_filters[key]
                        kf.predict()
                        kf.update(np.array([x, y]))
                        pred_x, pred_y = int(kf.x[0]), int(kf.x[1])

                        # Append positions
                        frame_features.extend([pred_x, pred_y])

                    # Angles + status
                    for name, (a_idx, b_idx, c_idx) in ANGLE_JOINTS.items():
                        if all(joint_history.get(f'joint_{i}', []) for i in (a_idx, b_idx, c_idx)):
                            a = (pose_kalman_filters[f'joint_{a_idx}'].x[0], pose_kalman_filters[f'joint_{a_idx}'].x[1])
                            b = (pose_kalman_filters[f'joint_{b_idx}'].x[0], pose_kalman_filters[f'joint_{b_idx}'].x[1])
                            c = (pose_kalman_filters[f'joint_{c_idx}'].x[0], pose_kalman_filters[f'joint_{c_idx}'].x[1])

                            angle = calculate_angle(a, b, c)
                            delta_y = b[1] - joint_history.get(f'joint_{b_idx}', [(0,0)])[-1][1]
                            angle_velocity = 0
                            if name in angle_history and len(angle_history[name]) > 0:
                                angle_velocity = angle - angle_history[name][-1]

                            label = classify_joint_movement(delta_y, angle_velocity)

                            frame_features.extend([angle, status_history.get(name, ["STATIC"])[-1] if name in status_history else "STATIC"])
                        else:
                            frame_features.extend([0, "STATIC"])

                    # Replace status strings with numbers
                    status_map = {"UP": 1, "DOWN": -1, "STATIC": 0}
                    frame_features = [status_map.get(f, f) if isinstance(f, str) else f for f in frame_features]

                    sequence_features.append(frame_features)

        if sequence_features:
            all_sequences.append(sequence_features)
            all_labels.append(action)

# Convert to numpy arrays
X = np.array(all_sequences, dtype=object) 
y = np.array(all_labels)

np.save("X_sequences.npy", X)
np.save("y_labels.npy", y)

print(f"Saved {len(X)} sequences with labels.")


In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PoseSequenceTransformer(nn.Module):
    def __init__(self, num_features, num_classes, d_model=128, nhead=8, num_layers=4, dim_feedforward=256, dropout=0.1):
        super(PoseSequenceTransformer, self).__init__()
        
        # Linear layer to map input features to d_model dimension
        self.input_fc = nn.Linear(num_features, d_model)
        
        # Positional encoding
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        
        # Transformer Encoder
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        # Classification head
        self.fc_out = nn.Linear(d_model, num_classes)

    def forward(self, src):
        src = self.input_fc(src)  # [B, T, F] -> [B, T, d_model]
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src)
        output = output.mean(dim=1)  # Global average pooling over time
        output = self.fc_out(output)
        return output

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-torch.log(torch.tensor(10000.0)) / d_model))
        pe = torch.zeros(max_len, d_model)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)  # [1, max_len, d_model]
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)


In [11]:
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.preprocessing import LabelEncoder
from torch.nn.utils.rnn import pad_sequence

class PoseSequenceDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = LabelEncoder().fit_transform(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        seq = torch.tensor(self.X[idx], dtype=torch.float32)  # [T, F]
        label = torch.tensor(self.y[idx], dtype=torch.long)
        return seq, label

def collate_fn(batch):
    sequences, labels = zip(*batch)
    padded_seqs = pad_sequence(sequences, batch_first=True, padding_value=0)
    labels = torch.stack(labels)
    return padded_seqs, labels


In [ ]:
from sklearn.model_selection import KFold
from torch.optim import AdamW
from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import accuracy_score

# Load preprocessed data
X = np.load("X_sequences.npy", allow_pickle=True)
y = np.load("y_labels.npy", allow_pickle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(type(X))          # Likely <class 'list'>
print(type(X[0]))
print(device)        # Works only if X[0] is ndarray

X = [np.array(seq) for seq in X]
num_features = X[0].shape[1]

num_classes = len(np.unique(y))

kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold = 1

for train_idx, val_idx in kf.split(X):
    print(f"\n===== Fold {fold} =====")
    X = np.array([np.array(seq) for seq in X], dtype=object)
    y = np.array(y)

    # Now this works:
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    # X_train, X_val = X[train_idx], X[val_idx]
    # y_train, y_val = y[train_idx], y[val_idx]

    train_dataset = PoseSequenceDataset(X_train, y_train)
    val_dataset = PoseSequenceDataset(X_val, y_val)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

    model = PoseSequenceTransformer(num_features=num_features, num_classes=num_classes).to(device)
    optimizer = AdamW(model.parameters(), lr=0.001)
    scheduler = StepLR(optimizer, step_size=10, gamma=0.5)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(50):  # could be 100
        model.train()
        train_loss = 0
        train_preds, train_targets = [], []

        for seqs, labels in train_loader:
            seqs, labels = seqs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(seqs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_preds.extend(outputs.argmax(dim=1).cpu().numpy())
            train_targets.extend(labels.cpu().numpy())

        scheduler.step()
        train_acc = accuracy_score(train_targets, train_preds)

        # Validation
        model.eval()
        val_preds, val_targets = [], []
        with torch.no_grad():
            for seqs, labels in val_loader:
                seqs, labels = seqs.to(device), labels.to(device)
                outputs = model(seqs)
                val_preds.extend(outputs.argmax(dim=1).cpu().numpy())
                val_targets.extend(labels.cpu().numpy())

        val_acc = accuracy_score(val_targets, val_preds)

        print(f"Epoch [{epoch+1}/50] - Loss: {train_loss/len(train_loader):.4f} - Train Acc: {train_acc:.4f} - Val Acc: {val_acc:.4f}")

    fold += 1


<class 'numpy.ndarray'>
<class 'list'>
cuda

===== Fold 1 =====
Epoch [1/50] - Loss: 1.8396 - Train Acc: 0.2728 - Val Acc: 0.3170
Epoch [2/50] - Loss: 1.5382 - Train Acc: 0.3676 - Val Acc: 0.3340
Epoch [3/50] - Loss: 1.4464 - Train Acc: 0.4030 - Val Acc: 0.3472
Epoch [4/50] - Loss: 1.4303 - Train Acc: 0.4143 - Val Acc: 0.3491
Epoch [5/50] - Loss: 1.3865 - Train Acc: 0.4502 - Val Acc: 0.3906
Epoch [6/50] - Loss: 1.3318 - Train Acc: 0.4620 - Val Acc: 0.4038
Epoch [7/50] - Loss: 1.3080 - Train Acc: 0.4587 - Val Acc: 0.4245
Epoch [8/50] - Loss: 1.3206 - Train Acc: 0.4714 - Val Acc: 0.4000
Epoch [9/50] - Loss: 1.2581 - Train Acc: 0.4922 - Val Acc: 0.4962
Epoch [10/50] - Loss: 1.2174 - Train Acc: 0.5300 - Val Acc: 0.4491
Epoch [11/50] - Loss: 1.1565 - Train Acc: 0.5517 - Val Acc: 0.5113
Epoch [12/50] - Loss: 1.1081 - Train Acc: 0.5639 - Val Acc: 0.5019
Epoch [13/50] - Loss: 1.0896 - Train Acc: 0.5805 - Val Acc: 0.5226
Epoch [14/50] - Loss: 1.0554 - Train Acc: 0.6083 - Val Acc: 0.5283
Epoch [

In [ ]:
torch.save(model.state_dict(), "pose_transformer.pth")


In [ ]:
# ===== Preprocess test videos =====
TEST_PATH = r"test"  # path to your test folder
actions_test = os.listdir(TEST_PATH)

pose_kalman_filters = {}
joint_history = {}
angle_history = {}
status_history = {}

test_sequences = []
test_labels = []

for action in actions_test:
    action_path = os.path.join(TEST_PATH, action)
    videos = [f for f in os.listdir(action_path) if f.lower().endswith(('.mp4', '.avi'))]

    for video_file in tqdm(videos, desc=f"Processing {action}"):
        video_path = os.path.join(action_path, video_file)
        frames = preprocess_video(video_path, background_subtraction=False)

        sequence_features = []
        with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
            for frame in frames:
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = pose.process(image)

                if results.pose_landmarks:
                    frame_features = []
                    for idx, lm in enumerate(results.pose_landmarks.landmark):
                        x, y = int(lm.x * frame.shape[1]), int(lm.y * frame.shape[0])
                        key = f'joint_{idx}'

                        if key not in pose_kalman_filters:
                            pose_kalman_filters[key] = create_kalman_filter()
                            pose_kalman_filters[key].x[:2] = np.array([[x], [y]])
                        kf = pose_kalman_filters[key]
                        kf.predict()
                        kf.update(np.array([x, y]))
                        pred_x, pred_y = int(kf.x[0]), int(kf.x[1])
                        frame_features.extend([pred_x, pred_y])

                    for name, (a_idx, b_idx, c_idx) in ANGLE_JOINTS.items():
                        if all(joint_history.get(f'joint_{i}', []) for i in (a_idx, b_idx, c_idx)):
                            a = (pose_kalman_filters[f'joint_{a_idx}'].x[0], pose_kalman_filters[f'joint_{a_idx}'].x[1])
                            b = (pose_kalman_filters[f'joint_{b_idx}'].x[0], pose_kalman_filters[f'joint_{b_idx}'].x[1])
                            c = (pose_kalman_filters[f'joint_{c_idx}'].x[0], pose_kalman_filters[f'joint_{c_idx}'].x[1])

                            angle = calculate_angle(a, b, c)
                            delta_y = b[1] - joint_history.get(f'joint_{b_idx}', [(0,0)])[-1][1]
                            angle_velocity = 0
                            if name in angle_history and len(angle_history[name]) > 0:
                                angle_velocity = angle - angle_history[name][-1]

                            label = classify_joint_movement(delta_y, angle_velocity)
                            frame_features.extend([angle, status_history.get(name, ["STATIC"])[-1] if name in status_history else "STATIC"])
                        else:
                            frame_features.extend([0, "STATIC"])

                    status_map = {"UP": 1, "DOWN": -1, "STATIC": 0}
                    frame_features = [status_map.get(f, f) if isinstance(f, str) else f for f in frame_features]
                    sequence_features.append(frame_features)

        if sequence_features:
            test_sequences.append(sequence_features)
            test_labels.append(action)

# Save test arrays
np.save("X_test.npy", np.array(test_sequences, dtype=object))
np.save("y_test.npy", np.array(test_labels))
print(f"Saved {len(test_sequences)} test sequences.")


Processing Fall Down:   0%|          | 0/177 [00:00<?, ?it/s]C:\Users\lenovo\AppData\Local\Temp\ipykernel_32584\3711796478.py:39: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_x, pred_y = int(kf.x[0]), int(kf.x[1])
Processing Walking: 100%|██████████| 285/285 [10:42<00:00,  2.25s/it]

Saved 1139 test sequences.


In [ ]:
# Load processed test data
X_test = np.load("X_test.npy", allow_pickle=True)
y_test = np.load("y_test.npy", allow_pickle=True)

# Match label encoding with training
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(np.load("y_labels.npy", allow_pickle=True))  # fit on train labels

y_test_encoded = label_encoder.transform(y_test)
X_test = [np.array(seq) for seq in X_test]

# Prepare DataLoader
test_dataset = PoseSequenceDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

# Load model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_features = X_test[0].shape[1]
num_classes = len(label_encoder.classes_)

model = PoseSequenceTransformer(num_features=num_features, num_classes=num_classes).to(device)
model.load_state_dict(torch.load("pose_transformer.pth"))  
model.eval()


PoseSequenceTransformer(
  (input_fc): Linear(in_features=78, out_features=128, bias=True)
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-3): 4 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=256, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=256, out_features=128, bias=True)
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (fc_out): Linear(in_features=128, out_features=7, bias=True)
)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

all_preds, all_targets = [], []

with torch.no_grad():
    for seqs, labels in test_loader:
        seqs, labels = seqs.to(device), labels.to(device)
        outputs = model(seqs)
        preds = outputs.argmax(dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_targets.extend(labels.cpu().numpy())

test_acc = accuracy_score(all_targets, all_preds)
print(f"Test Accuracy: {test_acc:.4f}")
print("\nClassification Report:")
print(classification_report(all_targets, all_preds, target_names=label_encoder.classes_))


Test Accuracy: 0.5733

Classification Report:
              precision    recall  f1-score   support

   Fall Down       0.66      0.53      0.59       169
  Lying Down       0.62      0.83      0.71       142
    Sit down       0.43      0.26      0.32        77
     Sitting       0.33      0.35      0.34       120
    Stand up       0.47      0.39      0.43       119
    Standing       0.54      0.66      0.59       229
     Walking       0.73      0.65      0.69       283

    accuracy                           0.57      1139
   macro avg       0.54      0.53      0.52      1139
weighted avg       0.58      0.57      0.57      1139



In [ ]:
from sklearn.metrics import accuracy_score, classification_report

all_preds, all_targets = [], []

with torch.no_grad():
    for seqs, labels in test_loader:
        seqs, labels = seqs.to(device), labels.to(device)
        outputs = model(seqs)
        preds = outputs.argmax(dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_targets.extend(labels.cpu().numpy())

test_acc = accuracy_score(all_targets, all_preds)
print(f"Test Accuracy: {test_acc:.4f}")
print("\nClassification Report:")
print(classification_report(all_targets, all_preds, target_names=label_encoder.classes_))


In [ ]:
from sklearn.model_selection import KFold
from torch.optim import AdamW
from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import accuracy_score

# Load preprocessed data
X = np.load("X_sequences.npy", allow_pickle=True)
y = np.load("y_labels.npy", allow_pickle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(type(X))          # Likely <class 'list'>
print(type(X[0]))
print(device)        # Works only if X[0] is ndarray

X = [np.array(seq) for seq in X]
num_features = X[0].shape[1]

num_classes = len(np.unique(y))

kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold = 1

for train_idx, val_idx in kf.split(X):
    print(f"\n===== Fold {fold} =====")
    X = np.array([np.array(seq) for seq in X], dtype=object)
    y = np.array(y)

    # Now this works:
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    # X_train, X_val = X[train_idx], X[val_idx]
    # y_train, y_val = y[train_idx], y[val_idx]

    train_dataset = PoseSequenceDataset(X_train, y_train)
    val_dataset = PoseSequenceDataset(X_val, y_val)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

    model1 = PoseSequenceTransformer(num_features=num_features, num_classes=num_classes).to(device)
    optimizer = AdamW(model1.parameters(), lr=0.001)
    scheduler = StepLR(optimizer, step_size=10, gamma=0.5)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(80):  # could be 100
        model1.train()
        train_loss = 0
        train_preds, train_targets = [], []

        for seqs, labels in train_loader:
            seqs, labels = seqs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model1(seqs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_preds.extend(outputs.argmax(dim=1).cpu().numpy())
            train_targets.extend(labels.cpu().numpy())

        scheduler.step()
        train_acc = accuracy_score(train_targets, train_preds)

        # Validation
        model1.eval()
        val_preds, val_targets = [], []
        with torch.no_grad():
            for seqs, labels in val_loader:
                seqs, labels = seqs.to(device), labels.to(device)
                outputs = model1(seqs)
                val_preds.extend(outputs.argmax(dim=1).cpu().numpy())
                val_targets.extend(labels.cpu().numpy())

        val_acc = accuracy_score(val_targets, val_preds)

        print(f"Epoch [{epoch+1}/80] - Loss: {train_loss/len(train_loader):.4f} - Train Acc: {train_acc:.4f} - Val Acc: {val_acc:.4f}")

    fold += 1


<class 'numpy.ndarray'>
<class 'list'>
cuda

===== Fold 1 =====
Epoch [1/80] - Loss: 1.7588 - Train Acc: 0.2827 - Val Acc: 0.3283
Epoch [2/80] - Loss: 1.5352 - Train Acc: 0.3709 - Val Acc: 0.3075
Epoch [3/80] - Loss: 1.4797 - Train Acc: 0.3893 - Val Acc: 0.3585
Epoch [4/80] - Loss: 1.4358 - Train Acc: 0.4210 - Val Acc: 0.3434
Epoch [5/80] - Loss: 1.4128 - Train Acc: 0.4422 - Val Acc: 0.4057
Epoch [6/80] - Loss: 1.3178 - Train Acc: 0.4700 - Val Acc: 0.4226
Epoch [7/80] - Loss: 1.2923 - Train Acc: 0.4733 - Val Acc: 0.3811
Epoch [8/80] - Loss: 1.2763 - Train Acc: 0.4889 - Val Acc: 0.4019
Epoch [9/80] - Loss: 1.3074 - Train Acc: 0.4757 - Val Acc: 0.4075
Epoch [10/80] - Loss: 1.2976 - Train Acc: 0.4799 - Val Acc: 0.4113
Epoch [11/80] - Loss: 1.2119 - Train Acc: 0.5229 - Val Acc: 0.4491
Epoch [12/80] - Loss: 1.1785 - Train Acc: 0.5503 - Val Acc: 0.5000
Epoch [13/80] - Loss: 1.1447 - Train Acc: 0.5521 - Val Acc: 0.5019
Epoch [14/80] - Loss: 1.1405 - Train Acc: 0.5493 - Val Acc: 0.4906
Epoch [

In [ ]:
torch.save(model1.state_dict(), "pose_transformer1.pth")

In [ ]:
import torch
from torch.utils.data import DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

X_test = np.load("X_test.npy", allow_pickle=True)
y_test = np.load("y_test.npy", allow_pickle=True)

X_test = [np.array(seq) for seq in X_test]
num_features = X_test[0].shape[1]  
num_classes = len(np.unique(y_test))

label_encoder = LabelEncoder()
y_test_encoded = label_encoder.fit_transform(y_test)

test_dataset = PoseSequenceDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

model1 = PoseSequenceTransformer(num_features=num_features, num_classes=num_classes).to(device)

model1.load_state_dict(torch.load("pose_transformer1.pth", map_location=device))
model1.eval()

all_preds, all_targets = [], []

with torch.no_grad():
    for seqs, labels in test_loader:
        seqs, labels = seqs.to(device), labels.to(device)
        outputs = model1(seqs)
        preds = outputs.argmax(dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_targets.extend(labels.cpu().numpy())

test_acc = accuracy_score(all_targets, all_preds)
print(f"Test Accuracy: {test_acc:.4f}")
print("\nClassification Report:")
print(classification_report(all_targets, all_preds, target_names=label_encoder.classes_))


Test Accuracy: 0.6172

Classification Report:
              precision    recall  f1-score   support

   Fall Down       0.71      0.54      0.62       169
  Lying Down       0.64      0.87      0.74       142
    Sit down       0.58      0.36      0.45        77
     Sitting       0.43      0.61      0.51       120
    Stand up       0.53      0.45      0.48       119
    Standing       0.60      0.69      0.64       229
     Walking       0.74      0.62      0.68       283

    accuracy                           0.62      1139
   macro avg       0.61      0.59      0.59      1139
weighted avg       0.63      0.62      0.61      1139



In [12]:
import torch
import cv2
import mediapipe as mp
import numpy as np
from sklearn.preprocessing import LabelEncoder

# ===== Parameters =====
VIDEO_PATH = r"D:\fitz\fitzpatrick17k-main\pose\UCF50\GolfSwing\v_GolfSwing_g01_c01.avi"
  # your custom video
MODEL_PATH = "pose_transformer1.pth"
LABELS_PATH = "y_labels.npy"    # from training

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ===== Load label encoder from training labels =====
y_train = np.load(LABELS_PATH, allow_pickle=True)
label_encoder = LabelEncoder()
label_encoder.fit(y_train)

# ===== Mediapipe setup =====
mp_pose = mp.solutions.pose

# ===== Feature extraction for a single video =====
def extract_features_from_video(video_path):
    pose_kalman_filters = {}
    joint_history = {}
    angle_history = {}
    status_history = {}

    frames = preprocess_video(video_path, background_subtraction=False)
    sequence_features = []

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        for frame in frames:
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(image)

            if results.pose_landmarks:
                frame_features = []
                for idx, lm in enumerate(results.pose_landmarks.landmark):
                    x, y = int(lm.x * frame.shape[1]), int(lm.y * frame.shape[0])

                    key = f'joint_{idx}'
                    if key not in pose_kalman_filters:
                        pose_kalman_filters[key] = create_kalman_filter()
                        pose_kalman_filters[key].x[:2] = np.array([[x], [y]])
                    kf = pose_kalman_filters[key]
                    kf.predict()
                    kf.update(np.array([x, y]))
                    pred_x, pred_y = int(kf.x[0]), int(kf.x[1])

                    frame_features.extend([pred_x, pred_y])

                # Angles + status
                for name, (a_idx, b_idx, c_idx) in ANGLE_JOINTS.items():
                    if all(joint_history.get(f'joint_{i}', []) for i in (a_idx, b_idx, c_idx)):
                        a = (pose_kalman_filters[f'joint_{a_idx}'].x[0], pose_kalman_filters[f'joint_{a_idx}'].x[1])
                        b = (pose_kalman_filters[f'joint_{b_idx}'].x[0], pose_kalman_filters[f'joint_{b_idx}'].x[1])
                        c = (pose_kalman_filters[f'joint_{c_idx}'].x[0], pose_kalman_filters[f'joint_{c_idx}'].x[1])

                        angle = calculate_angle(a, b, c)
                        delta_y = b[1] - joint_history.get(f'joint_{b_idx}', [(0,0)])[-1][1]
                        angle_velocity = 0
                        if name in angle_history and len(angle_history[name]) > 0:
                            angle_velocity = angle - angle_history[name][-1]

                        label = classify_joint_movement(delta_y, angle_velocity)
                        frame_features.extend([angle, {"UP":1, "DOWN":-1, "STATIC":0}[label]])
                    else:
                        frame_features.extend([0, 0])

                sequence_features.append(frame_features)

    return np.array(sequence_features, dtype=np.float32)  # [T, 78]

# ===== Extract features for this video =====
features = extract_features_from_video(VIDEO_PATH)

# ===== Load model =====
num_features = features.shape[1]
num_classes = len(label_encoder.classes_)
model = PoseSequenceTransformer(num_features=num_features, num_classes=num_classes).to(device)
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.eval()

# ===== Prepare for prediction =====
seq_tensor = torch.tensor(features, dtype=torch.float32).unsqueeze(0).to(device)  # [1, T, 78]

with torch.no_grad():
    output = model(seq_tensor)
    pred_class = torch.argmax(output, dim=1).cpu().item()
    pred_label = label_encoder.inverse_transform([pred_class])[0]

print(f"Predicted Action: {pred_label}")


C:\Users\lenovo\AppData\Local\Temp\ipykernel_20092\3977832452.py:50: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_x, pred_y = int(kf.x[0]), int(kf.x[1])


Predicted Action: Sitting


In [ ]:
import cv2
import torch
import mediapipe as mp
import numpy as np
from sklearn.preprocessing import LabelEncoder


VIDEO_PATH = r"D:\fitz\fitzpatrick17k-main\pose\UCF50\video_1.avi"
MODEL_PATH = "pose_transformer1.pth"
LABELS_PATH = "y_labels.npy"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose


y_train = np.load(LABELS_PATH, allow_pickle=True)
label_encoder = LabelEncoder()
label_encoder.fit(y_train)


num_features = 78  
num_classes = len(label_encoder.classes_)
model = PoseSequenceTransformer(num_features=num_features, num_classes=num_classes).to(device)
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.eval()


pose_kalman_filters = {}
joint_history = {}
angle_history = {}
status_history = {}

frames = preprocess_video(VIDEO_PATH, background_subtraction=False)
video_writer = cv2.VideoWriter("prediction_output.avi", cv2.VideoWriter_fourcc(*'XVID'), 20,
                               (frames[0].shape[1], frames[0].shape[0]))

sequence_features = []

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    for frame in frames:
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
        )

        if results.pose_landmarks:
            frame_features = []
            for idx, lm in enumerate(results.pose_landmarks.landmark):
                x, y = int(lm.x * image.shape[1]), int(lm.y * image.shape[0])
                key = f'joint_{idx}'

                if key not in pose_kalman_filters:
                    pose_kalman_filters[key] = create_kalman_filter()
                    pose_kalman_filters[key].x[:2] = np.array([[x], [y]])
                kf = pose_kalman_filters[key]
                kf.predict()
                kf.update(np.array([x, y]))
                pred_x, pred_y = int(kf.x[0]), int(kf.x[1])

                if key not in joint_history:
                    joint_history[key] = []
                joint_history[key].append((pred_x, pred_y))

                frame_features.extend([pred_x, pred_y])

            # Angles + status
            for name, (a_idx, b_idx, c_idx) in ANGLE_JOINTS.items():
                if all(len(joint_history.get(f'joint_{i}', [])) >= 2 for i in (a_idx, b_idx, c_idx)):
                    a = joint_history[f'joint_{a_idx}'][-1]
                    b = joint_history[f'joint_{b_idx}'][-1]
                    c = joint_history[f'joint_{c_idx}'][-1]

                    angle = calculate_angle(a, b, c)
                    angle_history.setdefault(name, []).append(angle)

                    delta_y = b[1] - joint_history[f'joint_{b_idx}'][-2][1]
                    angle_velocity = angle - angle_history[name][-2] if len(angle_history[name]) > 1 else 0
                    label = classify_joint_movement(delta_y, angle_velocity)

                    status_history.setdefault(name, []).append(label)

                    frame_features.extend([angle, {"UP":1, "DOWN":-1, "STATIC":0, "TRANSITION":1}[label]])
                else:
                    frame_features.extend([0, 0])

            sequence_features.append(frame_features)

        # ==== Prediction every 10 frames ====
        if len(sequence_features) >= 10:
            feat_array = np.array(sequence_features, dtype=np.float32)
            seq_tensor = torch.tensor(feat_array, dtype=torch.float32).unsqueeze(0).to(device)
            with torch.no_grad():
                output = model(seq_tensor)
                pred_class = torch.argmax(output, dim=1).cpu().item()
                pred_label = label_encoder.inverse_transform([pred_class])[0]
            cv2.putText(image, f"Action: {pred_label}",
                        (30, 70), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 2)

        video_writer.write(image)
        cv2.imshow('Pose Tracking + Prediction', image)

        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

cv2.destroyAllWindows()
video_writer.release()


# Final code for activity tracking

In [ ]:
import cv2
import torch
import mediapipe as mp
import numpy as np
from sklearn.preprocessing import LabelEncoder
import csv

VIDEO_PATH = r"D:\fitz\fitzpatrick17k-main\pose\UCF50\video_1.avi"
MODEL_PATH = "pose_transformer1.pth"
LABELS_PATH = "y_labels.npy"


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose


def draw_trajectories(image, joint_history, max_len=20):
    """Draw trailing lines showing movement history for each joint."""
    for key, points in joint_history.items():
        if len(points) < 2:
            continue
        color = (0, 255, 255)  # Cyan color for trails
        for i in range(max(0, len(points) - max_len), len(points) - 1):
            cv2.line(image, points[i], points[i + 1], color, 2)


def save_csv_log(filename, angle_history, joint_history, status_history):
    """Save angle, joint positions, and status to CSV file."""
    with open(filename, mode='w', newline='') as csv_file:
        fieldnames = ['frame', 'joint', 'angle', 'x', 'y', 'status']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        frames = len(next(iter(angle_history.values()), []))
        for frame_idx in range(frames):
            for joint_name in ANGLE_JOINTS.keys():
                angle = angle_history.get(joint_name, [None]*frames)[frame_idx]
                # Use last known joint b (middle joint) position as example
                b_idx = ANGLE_JOINTS[joint_name][1]
                pos = joint_history.get(f'joint_{b_idx}', [(None, None)]*frames)[frame_idx]
                status = status_history.get(joint_name, [""]*frames)[frame_idx]
                writer.writerow({
                    'frame': frame_idx,
                    'joint': joint_name,
                    'angle': angle,
                    'x': pos[0],
                    'y': pos[1],
                    'status': status
                })

# ===== Load label encoder =====
y_train = np.load(LABELS_PATH, allow_pickle=True)
label_encoder = LabelEncoder()
label_encoder.fit(y_train)

# ===== Load trained model =====
num_features = 78  # Must match training
num_classes = len(label_encoder.classes_)
model = PoseSequenceTransformer(num_features=num_features, num_classes=num_classes).to(device)
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.eval()

# ===== Kalman filter + histories =====
pose_kalman_filters = {}
joint_history = {}
angle_history = {}
status_history = {}

# ===== Video setup =====
frames = preprocess_video(VIDEO_PATH, background_subtraction=False)
frame_width, frame_height = frames[0].shape[1], frames[0].shape[0]
fps = 20
video_writer = cv2.VideoWriter("prediction_output.avi", cv2.VideoWriter_fourcc(*'XVID'), fps,
                               (frame_width, frame_height))

sequence_features = []

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    for frame_idx, frame in enumerate(frames):
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Draw pose landmarks
        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
        )

        if results.pose_landmarks:
            frame_features = []
            for idx, lm in enumerate(results.pose_landmarks.landmark):
                x, y = int(lm.x * image.shape[1]), int(lm.y * image.shape[0])
                key = f'joint_{idx}'

                if key not in pose_kalman_filters:
                    pose_kalman_filters[key] = create_kalman_filter()
                    pose_kalman_filters[key].x[:2] = np.array([[x], [y]])
                kf = pose_kalman_filters[key]
                kf.predict()
                kf.update(np.array([x, y]))
                pred_x, pred_y = int(kf.x[0]), int(kf.x[1])

                if key not in joint_history:
                    joint_history[key] = []
                joint_history[key].append((pred_x, pred_y))

                frame_features.extend([pred_x, pred_y])

            # Angles + status + visualization
            for name, (a_idx, b_idx, c_idx) in ANGLE_JOINTS.items():
                if all(len(joint_history.get(f'joint_{i}', [])) >= 2 for i in (a_idx, b_idx, c_idx)):
                    a = joint_history[f'joint_{a_idx}'][-1]
                    b = joint_history[f'joint_{b_idx}'][-1]
                    c = joint_history[f'joint_{c_idx}'][-1]

                    angle = calculate_angle(a, b, c)
                    angle_history.setdefault(name, []).append(angle)

                    delta_y = b[1] - joint_history[f'joint_{b_idx}'][-2][1]
                    angle_velocity = angle - angle_history[name][-2] if len(angle_history[name]) > 1 else 0
                    label = classify_joint_movement(delta_y, angle_velocity) or "STATIC"

                    status_history.setdefault(name, []).append(label)

                    movement_value = {"UP": 1, "DOWN": -1, "STATIC": 0, "TRANSITION": 1}.get(label, 0)
                    frame_features.extend([angle, movement_value])

                    # Show joint angle + label on frame
                    cv2.putText(image, f"{name}: {label} {int(angle)}°",
                                (b[0] + 10, b[1] - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                                (0, 255, 255), 1)
                else:
                    frame_features.extend([0, 0])

            sequence_features.append(frame_features)

        # ==== Prediction every 10 frames ====
        if len(sequence_features) >= 10:
            feat_array = np.array(sequence_features, dtype=np.float32)
            seq_tensor = torch.tensor(feat_array, dtype=torch.float32).unsqueeze(0).to(device)
            with torch.no_grad():
                output = model(seq_tensor)
                pred_class = torch.argmax(output, dim=1).cpu().item()
                pred_label = label_encoder.inverse_transform([pred_class])[0]
            cv2.putText(image, f"Action: {pred_label}",
                        (30, 70), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 2)

        # Draw movement trajectories
        # draw_trajectories(image, joint_history)

        video_writer.write(image)
        cv2.imshow('Pose Tracking + Prediction', image)

        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

cv2.destroyAllWindows()

save_csv_log("joint_angle_status_log.csv", angle_history, joint_history, status_history)

video_writer.release()
